In [1]:
import sys
sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")
import os
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm

DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    FACTOR_MODEL_ESTIMATES,
    FIX_SET_OF_HALF_LIFES,
    SPECIFIC_RISK
)


In [2]:
import multiprocessing
from factor_model.risk_calculations.specific_risk import (
    generate_raw_specific_risk,
    generate_raw_portfolio_specific_risk
)
from factor_model.risk_calculations.core_universe_portfolio import generate_market_portfolio

In [3]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    full_specific_returns = pd.read_sql_query(f"SELECT * FROM specific_returns", conn)

In [4]:
dates = list(full_specific_returns["date"].unique())[1:]
FIX_SET_OF_HALF_LIFES = sorted(list(range(10, 3 * 365, 30)) + [365])
risk_calculation_parameters = {
    "correlation_half_life": 730,  # days
    "variance_half_life": 365,  # days
    "specific_risk_half_life": 365,
    "date": "2023-03-04",
    "minimum_history_spec_ret": 730
}

In [5]:
def multi_process_calc(temp_half_life):
    symbols = set(full_specific_returns["ticker"])
    risk_calculation_parameters = {"specific_risk_half_life": temp_half_life}
    risk_for_half_life = list()
    for symbol in tqdm(symbols):
        specific_returns_temp = full_specific_returns[
            (full_specific_returns["ticker"] == symbol)
        ].copy()
        ewma_temp = specific_returns_temp[["date"]].copy()
        ewma_temp[symbol] = (
            specific_returns_temp["specific_return"]
            .ewm(halflife=risk_calculation_parameters["specific_risk_half_life"])
            .std()
        )
        ewma_temp["symbol"] = symbol
        risk_for_half_life.append(ewma_temp.rename(columns={symbol:"specific_risk"}))
    return pd.concat(risk_for_half_life, axis=0)

In [6]:
symbols = set(full_specific_returns["ticker"])

all_core_spec_risk = {}

with multiprocessing.Pool(processes=4) as pool:
    # res_struct = pool.map(multi_process_calc, FIX_SET_OF_HALF_LIFES)
    res_struct = list(tqdm(pool.imap(multi_process_calc, FIX_SET_OF_HALF_LIFES)))
    for df, half_life  in zip(res_struct, FIX_SET_OF_HALF_LIFES):
        all_core_spec_risk[half_life] = df

0it [00:00, ?it/s]

In [ ]:
# symbols = set(full_specific_returns["ticker"])

# all_core_spec_risk = {}

# for temp_half_life in tqdm(FIX_SET_OF_HALF_LIFES):
#     risk_calculation_parameters["specific_risk_half_life"] = temp_half_life
#     risk_for_half_life = None
#     for symbol in list(symbols):
#         specific_returns_temp = full_specific_returns[
#             (full_specific_returns["ticker"] == symbol)
#         ].copy()
#         ewma_temp = specific_returns_temp[["date"]].copy()
#         ewma_temp[symbol] = (
#             specific_returns_temp["specific_return"]
#             .ewm(halflife=risk_calculation_parameters["specific_risk_half_life"])
#             .std()
#         )
#         if risk_for_half_life is None:
#             risk_for_half_life = ewma_temp.copy()
#         else:
#             risk_for_half_life = risk_for_half_life.merge(ewma_temp, how ="outer", on ="date")

#     all_core_spec_risk[temp_half_life] = risk_for_half_life

  0%|          | 0/38 [04:43<?, ?it/s]


KeyboardInterrupt: 

In [9]:
all_core_spec_risk = {}

for date in tqdm(dates[0:1]):
    # 1. gather exposures
    with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
        exposure = pd.read_sql_query(f"SELECT * FROM exposures where date = '{date}'", conn)
    # 2. identify universe_info
    market_portfolio = generate_market_portfolio(exposure)
    # 3. generate output
    core_specific_risk_df = pd.DataFrame({
        "date" : [date]
    })
    for temp_half_life in FIX_SET_OF_HALF_LIFES:
        risk_calculation_parameters["specific_risk_half_life"] = temp_half_life
        risk_calculation_parameters["date"] = date
        spec_std, hist_len = generate_raw_specific_risk(full_specific_returns, risk_calculation_parameters, market_portfolio)
        core_specific_risk_df[f"core_spec_risk_{temp_half_life}"] = generate_raw_portfolio_specific_risk(spec_std, market_portfolio)
    all_core_spec_risk.append(core_specific_risk_df)

  0%|          | 0/1 [01:52<?, ?it/s]


KeyboardInterrupt: 